In [2]:
import pandas as pd
import glob as g

# audio deep learning

shamelessly copied from https://towardsdatascience.com/audio-deep-learning-made-simple-sound-classification-step-by-step-cebc936bbe5

In [15]:
class_ids = [0,1,2]
#violin is 0
#trumpet is 1
#guitar is 2

paths_violin = g.glob('data/violin/*')
paths_guitar = g.glob('data/guitar/*')
paths_trumpet = g.glob('data/trumpet/*')

num_samples = len(paths_trumpet) + len(paths_guitar) + len(paths_violin)

paths_all = paths_violin + paths_guitar + paths_trumpet
ids_all = [0]*len(paths_trumpet) + [1]*len(paths_guitar) + [2]*len(paths_violin)

In [20]:
meta = pd.DataFrame({'path':paths_all,'class':ids_all})

In [21]:
meta

,path,class
0,data/violin/violin_D5_15_pianissimo_arco-norma...,0
1,data/violin/violin_Gs4_phrase_forte_arco-marte...,0
2,data/violin/violin_Gs4_1_fortissimo_arco-norma...,0
3,data/violin/violin_B5_025_mezzo-forte_pizz-nor...,0
4,data/violin/violin_As5_05_piano_arco-normal.mp3,0
...,...,...
2088,data/trumpet/trumpet_D4_1_pianissimo_normal.mp3,2
2089,data/trumpet/trumpet_E4_phrase_mezzo-forte_tri...,2
2090,data/trumpet/trumpet_Ds4_1_forte_normal.mp3,2
2091,data/trumpet/trumpet_D4_long_pianissimo_normal...,2
